In [ ]:
#This is a complete code document for performing LDA on cleaned tweet data

In [1]:
import pandas as pd
import os
from configparser import ConfigParser
import numpy as np
import spacy
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation #importing the relevant library
from sklearn.preprocessing import normalize
from collections import Counter

In [4]:
df_tweets = pd.read_csv('rcd_002_LDA.csv')

In [5]:
df_tweets.head()

Unnamed: 0            ID      Location  \
0           0  4.653654e+09      new york   
1           1  1.804594e+08       florida   
2           2  9.259447e+07  north dakota   
3           3  1.815191e+06      new york   
4           4  1.052475e+18        hawaii   

                              Further_Cleaned_Text\r  LDA_Topic  
0  heartbreaking covid19 washington attention dif...          1  
1  op nurse hospital treat covid19 yesterday hosp...          3  
2                             present covid19 tune\r          4  
3                            ffs tax people misery\r          5  
4  coronavirus diary take year away abuser shelte...          3

In [6]:
df_tweets = df_tweets[['ID','Location','Further_Cleaned_Text\r']]
df_tweets.head()

ID      Location  \
0  4.653654e+09      new york   
1  1.804594e+08       florida   
2  9.259447e+07  north dakota   
3  1.815191e+06      new york   
4  1.052475e+18        hawaii   

                              Further_Cleaned_Text\r  
0  heartbreaking covid19 washington attention dif...  
1  op nurse hospital treat covid19 yesterday hosp...  
2                             present covid19 tune\r  
3                            ffs tax people misery\r  
4  coronavirus diary take year away abuser shelte...

In [7]:
df_tweets.columns

Index(['ID', 'Location', 'Further_Cleaned_Text\r'], dtype='object')

In [8]:
cleaned_text = df_tweets['Further_Cleaned_Text\r']

In [9]:
df_tweets['Further_Cleaned_Text\r'].head()

0    heartbreaking covid19 washington attention dif...
1    op nurse hospital treat covid19 yesterday hosp...
2                               present covid19 tune\r
3                              ffs tax people misery\r
4    coronavirus diary take year away abuser shelte...
Name: Further_Cleaned_Text\r, dtype: object

In [10]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

dtm = cv.fit_transform(df_tweets['Further_Cleaned_Text\r'].apply(lambda x: np.str_(x)))

In [11]:
LDA = LatentDirichletAllocation(n_components=3,random_state=42) #3 topics, random_state = 42 for constant output

In [12]:
LDA.fit(dtm) 


LatentDirichletAllocation(n_components=3, random_state=42)

In [13]:
len(cv.get_feature_names()) # this is a list of stored/unique words

185864

In [14]:
import random

for i in range(10): # selecting 10 random words from the list of unique words
    random_word_id = random.randint(0,185864)
    print(cv.get_feature_names()[random_word_id])

2k
youtubedouble
coronabro
schs
cbts
mutations
justiceforniawilson
safel
spokesbarbie
burnetta


In [15]:
len(LDA.components_) #type is a numpy array containing probabilities for each word (7 by 38669)

3

In [16]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['positive', 'day', 'state', 'report', '000', 'covid19', 'people', 'trump', 'new', 'case', 'test', 'death', 'coronavirus', '19', 'covid']


THE TOP 15 WORDS FOR TOPIC #1
['crisis', 'government', 'state', 'worker', 'need', 'health', 'people', 'work', 'help', '19', 'pandemic', 'amp', 'covid', 'covid19', 'coronavirus']


THE TOP 15 WORDS FOR TOPIC #2
['doctor', 'think', 'say', 'dr', '19', 'trump', 'mask', 'know', 'like', 'people', 'coronavirus', 'covid', 'face', 'virus', 'corona']




In [17]:
topic_results = LDA.transform(dtm)

In [18]:
topic_results.shape # 404289 questions, and 10 topics

(3518296, 3)

In [19]:
df_tweets['LDA_Topic'] = topic_results.argmax(axis=1)

In [20]:
df_tweets.head(10)

ID        Location  \
0  4.653654e+09        new york   
1  1.804594e+08         florida   
2  9.259447e+07    north dakota   
3  1.815191e+06        new york   
4  1.052475e+18          hawaii   
5  1.960223e+07           texas   
6  4.240471e+08  south carolina   
7  8.379316e+17  north carolina   
8  1.222721e+18      california   
9  1.665635e+07  south carolina   

                              Further_Cleaned_Text\r  LDA_Topic  
0  heartbreaking covid19 washington attention dif...          1  
1  op nurse hospital treat covid19 yesterday hosp...          1  
2                             present covid19 tune\r          1  
3                            ffs tax people misery\r          2  
4  coronavirus diary take year away abuser shelte...          0  
5  week    declare covid19 national emergency    ...          0  
6  economic virus work coronavirus conversation d...          1  
7  happen petty democratic tyrant population try ...          1  
8     race go to kill fast coronavirus republicans\r          2  
9  6 death 205 covid19 case dallas county    day ...          0

In [21]:
df_tweets.to_csv('rcd_002_LDA.csv')